# Running SAR on MovieLens (Python, CPU)

SAR is a fast scalable adaptive algorithm for personalized recommendations based on user transaction history and item descriptions. It produces easily explainable / interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. 

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users who have interacted with one item are also likely to have interacted with another. A user has an ***affinity*** to an item if they have interacted with it in the past.

### SAR has the following advantages:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time. 
- Fast scoring, only involving multiplication of the similarity matric with an affinity vector

### And the following drawbacks:
- Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
- It's memory-hungry, requiring the creation of an $m×m$ square matrix (where $m$ is the number of items).
- Only outputs a SAR score (rather than an estimated rating).

This notebook provides an example of how to utilize and evaluate SAR in Python on a CPU.

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")

from utilities.recommender.sar.sar_singlenode import SARSingleNodeReference
from utilities.dataset.url_utils import maybe_download
from utilities.dataset.python_splitters import python_random_split
from utilities.evaluation.python_evaluation import PythonRatingEvaluation, PythonRankingEvaluation

import itertools
import pandas as pd

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version: 0.23.0


### 1. Download the MovieLens dataset

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>,<Time>,[<Event Type>], [<Event Weight>]`. 

Each row represents a single interaction between a user and an item. The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies - we will use it for the rest of the example.

In [2]:
filepath = maybe_download("http://files.grouplens.org/datasets/movielens/ml-100k/u.data", "ml-100k.data")

In [3]:
data = pd.read_csv(filepath, sep="\t", names=["UserId", "MovieId", "Rating", "Timestamp"])
data.head()

,UserId,MovieId,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### 2. Split the data using the python random splitter provided in utilities:

We utilize the provided `python_random_split` function to split into `train` and `test` datasets randomly at a 80/20 ratio.

In [4]:
train, test = python_random_split(data)

### 3. Instantiate the SAR algorithm and set the index

In order to use SAR, we need to hash users and items and set an index in order for SAR matrix operations to work. First we instantiate SAR using the reference implementation provided in `SARSingleNodeReference`:

In [5]:
header = {
        "col_user": "UserId",
        "col_item": "MovieId",
        "col_rating": "Rating",
        "col_timestamp": "Timestamp",
    }

model = SARSingleNodeReference(
                remove_seen=True, similarity_type="jaccard", 
                time_decay_coefficient=30, time_now=None, timedecay_formula=True, **header
            )

In [6]:
unique_users = data["UserId"].unique()
unique_items = data["MovieId"].unique()

We will hash users and items to smaller continuous space.
This is an ordered set - it's discrete, but contiguous.
This helps keep the matrices we keep in memory as small as possible.

In [7]:
enumerate_items_1, enumerate_items_2 = itertools.tee(enumerate(unique_items))
enumerate_users_1, enumerate_users_2 = itertools.tee(enumerate(unique_users))
item_map_dict = {x: i for i, x in enumerate_items_1}
user_map_dict = {x: i for i, x in enumerate_users_1}

The reverse of the dictionary above - array index to actual ID


In [8]:
index2user = dict(enumerate_users_2)
index2item = dict(enumerate_items_2)

We need to index the train and test sets for SAR matrix operations to work

In [9]:
model.set_index(unique_users, unique_items, user_map_dict, item_map_dict, index2user, index2item)

### 4. Train the SAR model on our training data, and get the top-k recommendations for our testing data

SAR first computes an item-to-item ***co-occurence matrix***. Co-occurence represents the number of times two items appear together for any given user. Once we have the co-occurence matrix, we compute an ***item similarity matrix*** by rescaling the cooccurences by a given metric (Jaccard similarity in this example). 

We also compute an ***affinity matrix*** to capture the strength of the relationship between each user and each item. Affinity is driven by different types (like *rating* or *viewing* a movie), and by the time of the event. 

Personalized recommendations are achieved by multiplying the affinity matrix $A$ and the similarity matrix $S$. The result is a ***recommendation score matrix*** $R$. We do not compute the full matrix $R$ to reduce memory requirements: we compute the ***top-k*** results for each user in the `recommend_k_items` function seen below.

A full walkthrough of the SAR algorithm can be found [here](../02_modeling/sar_educational_walkthrough.ipynb).

In [10]:
model.fit(train)
top_k = model.recommend_k_items(test)

INFO:utilities.recommender.sar.sar_singlenode:Collecting user affinity matrix...
INFO:utilities.recommender.sar.sar_singlenode:Calculating time-decayed affinities...
../..\utilities\recommender\sar\sar_singlenode.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["exponential"] = expo_fun(df[self.col_timestamp].values)
../..\utilities\recommender\sar\sar_singlenode.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["rating_exponential"] = df[self.col_rating] * df["exponential"]
INFO:utilities.recommender.sar.sar_singlenode:Creat

In [11]:
# TODO: remove this call when the model returns same type as input
top_k['UserId'] = pd.to_numeric(top_k['UserId'])
top_k['MovieId'] = pd.to_numeric(top_k['MovieId'])

In [12]:
display(top_k.head())

,UserId,MovieId,prediction
5787,796,234,155.419422
5786,796,216,154.755182
5789,796,174,154.093210
5788,796,566,153.102244
5785,796,79,152.824891


### 5. Evaluate how well SAR performs 

We evaluate how well SAR performs with for a few common ranking metrics provided in the `PythonRankingEvaluation` classs in utilities. We will consider the Mean Average Precision (MAP), Normalized Discounted Cumalative Gain (NDCG), Precision, and Recall for the top-k items per user we computed with SAR.

In [13]:
rank_eval = PythonRankingEvaluation(test, top_k, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="prediction", 
                                    relevancy_method="top_k")

In [14]:
print("Model:\t" + model.model_str,
      "Top K:\t%d" % rank_eval.top_k,
      "MAP@k:\t%f" % rank_eval.map_at_k(),
      "NDCG@k:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@k:\t%f" % rank_eval.precision_at_k(),
      "Recall@k:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	sar_ref
Top K:	10
MAP@k:	0.105815
NDCG@k:	0.373197
Precision@k:	0.326617
Recall@k:	0.175957
